In [1]:
eicu_path = '/scratch/wiensj_root/wiensj/shared_data/datasets/eicu-2.0/'
data_path = '/scratch/wiensj_root/wiensj/shared_data/FIDDLE_project/'

import pandas as pd
import numpy as np
from tqdm import tqdm

In [4]:
examples = pd.read_csv(data_path + 'extracted/icustays.csv')
labels = pd.read_csv(data_path + 'labels/Shock.csv')

In [5]:
print("original pupulation: ", examples['ICUStayID'].nunique() )

original pupulation:  200859


In [6]:
# Remove non-adults
min_age = 18
max_age = np.inf # no max age
examples = examples[(examples.age >= min_age) & (examples.age <= max_age)]
print('Exclude non-adults', examples['ICUStayID'].nunique())

Exclude non-adults 200234


In [7]:
hospitals = sorted(examples['hospitalid'].unique())

In [8]:
len(hospitals)

208

In [9]:
df_I = pd.read_pickle(data_path + 'extracted/infusionDrug.pickle')
df_M = pd.read_parquet(data_path + 'extracted/medication.parquet')

In [10]:
IDs = sorted(set(
    list(df_I['ID'].unique()) + \
    list(df_M['ID'].unique())
))

In [11]:
len(IDs)

176120

In [12]:
hospitals_ARF = sorted(examples.loc[examples['ICUStayID'].isin(IDs), 'hospitalid'].unique())

In [13]:
len(hospitals_ARF) # hospitals to keep

201

In [14]:
examples = examples[examples['hospitalid'].isin(hospitals_ARF)]

In [15]:
print('Exclude hospitals without labels', examples['ICUStayID'].nunique())

Exclude hospitals without labels 198382


In [16]:
examples_ = examples.rename(columns={'ICUStayID': 'ID'})
mort = pd.read_csv(data_path + 'labels/mortality.csv')
examples_ = pd.merge(examples_, mort, on='ID', how='left')

In [18]:
len(examples_)

198382

In [17]:
task = 'Shock'
for T in [4.0, 12.0]:
    print('======')
    print('prediction time', T, 'hour')

    # Remove discharege and deaths before cutoff hour
    examples = examples_[examples_['ICUDischargeTime'] >= T * 60]
    print('Exclude deaths', examples['ID'].nunique())

    print('---')
    print('Outcome', task)

    pop = examples[['ID']] \
            .merge(labels[['ID', '{}_ONSET_TIME'.format(task)]], on='ID', how='left')
    pop = pop[(pop['{}_ONSET_TIME'.format(task)] >= T*60) | pop['{}_ONSET_TIME'.format(task)].isnull()]
    pop['{}_LABEL'.format(task)] = pop['{}_ONSET_TIME'.format(task)].notnull().astype(int)
    pop.to_csv(data_path + 'population/{}_{}h.csv'.format(task, T), index=False)

    print('Exclude onset', len(pop))

prediction time 4.0 hour
Exclude deaths 186416
---
Outcome Shock
Exclude onset 164333
prediction time 12.0 hour
Exclude deaths 170730
---
Outcome Shock
Exclude onset 144725
